# CMP-3004

# Computer Organization

## Spring 2023

# Homework 3

### 1. Use assembly to solve the following problems:

* Obtain the 50th Fibonacci number



In [8]:
	    .data
LC0:	.asciiz "%d\n"

	    .text
	    .globl		fib
fib:
	    addiu	$sp, $sp, -8	# se setea el puntero 
	    sw	    $ra, 0($sp)		# se guarda la dirección del return
	
	    # Prolog de la función
	    sw	    $fp, -4($sp)	# se guarda el puntero anterior
	    move	$fp, $sp		# se setea el nuevo puntero
	
	    # Cuerpo de la función
	    lw	    $a0, 0($fp)		# se carga el valor de n
	    li	    $v0, 1			# se carga el valor 1 en v0
	    slt	    $v1, $a0, $v0	# se compara n con 1
	    bne	    $v1, $zero, L1	# se salta si n > 1
	
	    # Caso base: n <= 1
	    move	$v0, $a0		# Retornar n
	    j	fib_exit		    # Saltar a la salida de la función
	
L1:
	    addiu	$a0, $a0, -1	# se disminuye n en 1
	    jal	    fib			    # se usa recursividad para fib(n-1)
	    move	$s0, $v0		# se guarda el resultado en s0
	
	    addiu	$a0, $a0, -1	# se disminuye n en 1
	    jal	    fib			    # se usa recursividad para fib(n-2)
	    add	    $v0, $s0, $v0	# se suma el resultado de fib(n-1) con fib(n-2)
	    j	    fib_exit		# saltar a la salida de la función

fib_exit:
	    
	    move	$sp, $fp		# se reinicia el puntero 
	    lw	    $ra, 0($sp)		# se reinicia la dirección del return
	    addiu	$sp, $sp, 8		# se ajusta el puntero
	
	    jr	    $ra			    # se retorna desde fib

	    .globl		main
main:
	    addiu	$sp, $sp, -8	# se ajusta el puntero
	    sw	    $ra, 0($sp)		# se guarda la dirección del return
	
	    # Prolog de la función
	    sw	    $fp, -4($sp)	# se guarda el puntero anterior
	    move	$fp, $sp		# se setea el nuevo puntero
	
	    # Cuerpo de la función
	    li	    $a0, 50			# se hace que n sea 50
	    jal	    fib			    # se llama a la función fib
	    move	$a0, $v0		# se mueve el resultado de fib a a0
	    jal	    print_integer	# se llama a la función print_integer
	
	    li	    $v0, 5			# número de llamada al sistema para leer un entero
	    syscall				    # leer un entero
	
	    move	$v0, $zero		# se hace que el valor del return sea 0
	
	    # Epílogo de la función
	    move	$sp, $fp		# Restaurar puntero de pila
	    lw	    $ra, 0($sp)		# Restaurar dirección de retorno
	    addiu	$sp, $sp, 8		# Ajustar puntero de pila
	
	    jr	    $ra			    # Retornar desde main

print_integer:
	    addiu	$sp, $sp, -8	# Ajustar puntero de pila
	    sw  	$ra, 0($sp)		# Guardar dirección de retorno
	
	    # Imprimir entero
	    move	$t0, $a0		# Mover el entero a $t0

Line 2: Invalid argument: "%d\n"

* Find out if a given year is leap

the following code was tested in mars 4.5 and it worked well

In [13]:
.data
leapyear:    .asciiz "Es bisiesto"
notleapyear:    .asciiz "No es bisiesto"

.text
.globl main

main:
    li $t0, 2005
    
    # Check if the year is a leap year
    rem $t1, $t0, 4          # Calculate t1 = year % 4
    bnez $t1, not_leap_year   # If t1 != 0, it's not a leap year
    
    rem $t1, $t0, 100        # Calculate t1 = year % 100
    beqz $t1, leap_year       # If t1 == 0, it's a leap year
    
    rem $t1, $t0, 400        # Calculate t1 = year % 400
    bnez $t1, not_leap_year   # If t1 != 0, it's not a leap year
    
leap_year:
    li $v0, 4                 # System call number for printing a string
    la $a0, leapyear               # Load address of the leap year message
    syscall                   # Print the message
    
    j exit
    
not_leap_year:
    li $v0, 4                 # System call number for printing a string
    la $a0, notleapyear               # Load address of the non-leap year message
    syscall                   # Print the message
    
    j exit
    
exit:
    li $v0, 10                # System call number for exit
    syscall                   # Terminate the program

Line 2: Invalid argument: "Es

* Given an array of integers, calculate the average

In [15]:
.data
array: .word 26, 92, 13, 2, 8  # Example array of integers
array_size: .word 5          # Size of the array
result_msg: .asciiz "The average is: "
format_float: .asciiz "%f"

.text
.globl main

main:
    # Calculate the sum of the array elements
    lw $t0, array_size         # Load the array size
    li $t1, 0                  # Initialize the sum to zero
    
    loop:
        beq $t0, $zero, calculate_average  # If array size is zero, calculate average
        
        lw $t2, array($t0)      # Load the array element
        add $t1, $t1, $t2       # Add the element to the sum
        sub $t0, $t0, 4         # Decrement the array size (assuming each element is 4 bytes)
        j loop

    calculate_average:
        # Calculate the average
        mtc1 $t1, $f0            # Move the sum to floating-point register $f0
        cvt.s.w $f0, $f0         # Convert the sum to single precision
        lwc1 $f1, array_size     # Load the array size to floating-point register $f1
        div.s $f2, $f0, $f1      # Divide the sum by the array size to get the average

        # Print the result
        li $v0, 2                # System call code 2 for printing float
        la $a0, result_msg       # Load the message for printing
        syscall

        li $v0, 2                # System call code 2 for printing float
        mov.s $f12, $f2          # Move the average to floating-point register $f12
        syscall

    # Exit the program
    li $v0, 10
    syscall


Line 2: Invalid argument: #

* Write a program to transform from Celsius to Fahrenheit and vice versa

In [14]:
.data
prompt_fahrenheit: .asciiz "\n Enter the temperature in Fahrenheit: "
format_float: .asciiz "%f"
result_celsius: .asciiz "\n The temperature %.2f in Fahrenheit is %.2f in Celsius\n"
prompt_celsius: .asciiz "\n Enter the temperature in Celsius: "
result_fahrenheit: .asciiz "\n The temperature %.2f in Celsius is %.2f in Fahrenheit\n"
fahrenheit_constant: .float 32.0
celsius_constant1: .float 0.5556
celsius_constant: .float 1.8

.text
.globl main

main:
    # Print prompt for Fahrenheit temperature
    li $v0, 4
    la $a0, prompt_fahrenheit
    syscall
    
    # Read Fahrenheit temperature
    li $v0, 6
    syscall
    mov.s $f12, $f0
    
    # Load Fahrenheit constant from memory
    l.s $f10, fahrenheit_constant
    
    # Convert Fahrenheit to Celsius
    sub.s $f1, $f12, $f10
    l.s $f11, celsius_constant1
    mul.s $f2, $f1, $f11
    
    # Print result
    li $v0, 2
    la $a0, result_celsius
    mov.s $f12, $f2
    syscall
    
    # Print prompt for Celsius temperature
    li $v0, 4
    la $a0, prompt_celsius
    syscall
    
    # Read Celsius temperature
    li $v0, 6
    syscall
    mov.s $f12, $f0
    
    # Load Celsius constant from memory
    l.s $f10, celsius_constant
    
    # Convert Celsius to Fahrenheit
    mul.s $f1, $f12, $f10
    l.s $f11, fahrenheit_constant
    add.s $f2, $f1, $f11
    
    # Print result
    li $v0, 2
    la $a0, result_fahrenheit
    mov.s $f12, $f2
    syscall
    
    # Exit the program
    li $v0, 10
    syscall

Line 2: Invalid argument: "\n

### 2. For the following memory space, what would it look like after executing the assembly code below:

* `LOAD 14` = this instruction is loading the value of the address $14$ $(7)$ into a register.
* `ADD (12)` =  this instruction is adding the value of the address $12$ $(5)$ to the value previously stored in the register $(7)$. As a result, this instruction is making the operation: $5+7$
* `STORE 12` = this instruction is storing the resulting value of the previous instruction $(5+7)$ into address $12$. So, now address $12$ is not $5$, but $12$. 

All the other contents of the other addresses are still the same. 

### 3. Implement a function named `abs_diff` that calculates the absolute value of the difference of two inputs `a` and `b` (i.e., |a-b|), and get the assembly code output

Assembly out put obtained after using command `gcc -O1 -S ./abs.c`, where abs.c is the c program of the asked function:

In [ ]:
	.file	"abs.c"
	.text
	.globl	abs_diff
    .type	abs_diff, @function
	.section .rdata,"dr"
.LC0:
	.ascii "Enter a: \0"
.LC1:
	.ascii "%d\0"
.LC2:
	.ascii "Enter b: \0"
	.align 8
.LC3:
	.ascii "\12absolute difference between %d and %d is = %d\0"
	.text
	.globl	main
	.def	main;	.scl	2;	.type	32;	.endef
	.seh_proc	main
abs_diff:
	pushq	%rbx
	.seh_pushreg	%rbx
	subq	$48, %rsp
	.seh_stackalloc	48
	.seh_endprologue
	call	__main
	leaq	.LC0(%rip), %rcx
	call	printf
	leaq	44(%rsp), %rdx
	leaq	.LC1(%rip), %rbx
	movq	%rbx, %rcx
	call	scanf
	leaq	.LC2(%rip), %rcx
	call	printf
	leaq	40(%rsp), %rdx
	movq	%rbx, %rcx
	call	scanf
	movl	44(%rsp), %edx
	movl	40(%rsp), %r8d
	movl	%edx, %eax
	subl	%r8d, %eax
	movl	%r8d, %r9d
	subl	%edx, %r9d
	cmpl	%r8d, %edx
	cmovg	%eax, %r9d
	leaq	.LC3(%rip), %rcx
	call	printf
	movl	$0, %eax
	addq	$48, %rsp
	popq	%rbx
	ret
	.seh_endproc
	.ident	"GCC: (MinGW-W64 x86_64-ucrt-posix-seh, built by Brecht Sanders) 12.2.0"
	.def	printf;	.scl	2;	.type	32;	.endef
	.def	scanf;	.scl	2;	.type	32;	.endef

modified program in order to run it in mars mips simulator 4.5:

In [16]:
.data
prompt_first: .asciiz "Enter a: "
format_int: .asciiz "%d"
prompt_second: .asciiz "Enter b: "
result_msg: .asciiz "\nAbsolute difference between %d and %d is = %d\n"

.text
.globl abs_diff

abs_diff:
    # Prompt for the first number
    li $v0, 4
    la $a0, prompt_first
    syscall
    
    # Read the first number
    li $v0, 5
    syscall
    move $s0, $v0
    
    # Prompt for the second number
    li $v0, 4
    la $a0, prompt_second
    syscall
    
    # Read the second number
    li $v0, 5
    syscall
    move $s1, $v0
    
    # Calculate the difference
    sub $s2, $s0, $s1
    
    # Print the result
    li $v0, 1
    move $a0, $s0
    syscall
    
    li $v0, 1
    move $a0, $s1
    syscall
    
    li $v0, 1
    move $a0, $s2
    syscall
    
    # Exit the program
    li $v0, 10
    syscall

Line 2: Invalid argument: "Enter

### 4. What are the differences among sequential access, direct access, and random access?

References used: 
* https://www.geeksforgeeks.org/sequential-access-vs-direct-access-vs-random-access/
* https://www.techwalla.com/articles/direct-access-vs-sequential-access-vs-random-access



Sequential Access vs Direct Access: Streaming huge files or processing data sequentially are two examples of applications that would benefit from sequential access as opposed to direct access. Applications that need for speedy retrieval or change of particular data units, like querying or updating records in a database, are best served by direct access.


Sequential Access vs Random Access: Comparing sequential access to random access, sequential access is easier to implement and uses less overhead. While random access offers the freedom of immediately accessing any element, sequential access is efficient for activities that involve processing data in a particular order.

Direct Acess vs Random Access: Direct access entails using precise addresses or indexes to quickly access particular blocks or records of data. Random access enables quick accessibility to any data element regardless of its position, making it ideal for scenarios requiring fast and unpredictable access to data.



### 5. What common characteristics are shared by all RAID levels?

* Fault Tolerance: Several RAID levels provide fault tolerance by employing redundancy strategies like mirroring or parity data to safeguard data in the event of disk failures.

* Performance Enhancement: By providing parallel data access across several drives, RAID levels improve performance. This leads to an improvement in data transfer rate and I/O performance.

* Capacity Enhancement: RAID levels boost storage capacity by combining the capacity of several drives into a single logical volume.

* Data striping: To divide data into blocks and spread them over several drives, RAID levels use data striping. This improves read and write performance.

* Data Distribution: Different RAID levels use different algorithms or parity approaches to disperse data among disks.

references: 

* https://www.pcguide.com/ref/hdd/perf/raid/levels/index.htm
* https://www.geeksforgeeks.org/raid-redundant-array-of-independent-disks/

